In [1]:
!pip install rdflib > /dev/null
!pip install owlrl > /dev/null
!pip install pyshacl > /dev/null
!pip install autopep8 > /dev/null


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
from rdflib import *
from owlrl import *
from jinja2 import Template
import types
import pyshacl
import functools
import inspect
import autopep8
import json

In [3]:
tbox = Namespace('http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/tbox#')
abox = Namespace('http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/abox#')
dcat = Namespace('https://www.w3.org/ns/dcat#')
dcterms = Namespace('http://purl.org/dc/terms/')
tb = Namespace("http://www.semanticweb.org/acraf/ontologies/2021/0/SDM#")
odrl = Namespace("http://www.w3.org/ns/odrl/2/")

In [4]:
def add_jsonld_instances(graph, path):
    # Adds JSON-LD instances to the graph
    with open(path, 'r') as f:
        json_ld_data = json.loads(f.read())
        instances = Graph().parse(data=json_ld_data, format='json-ld')
        graph += instances
    
    return graph

# POLICYCHECKER TRANSLATOR

Given a Data Product, Output a DC IRs

In [5]:
import types
import autopep8
from jinja2 import Template
from rdflib import Graph
import time

class Implementation(type):
    def __init__(cls, name, bases, attrs):
        super().__init__(name, bases, attrs)
        cls.tg = cls._initialize_graph()
        cls.ex = Graph()
        cls.translation = None
    
    @staticmethod
    def _initialize_graph():
        tg = Graph()
        tg.bind("tb", "http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/tbox#")
        tg.bind("ab", "http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/abox#")
        tg = add_jsonld_instances(tg, "translations_mappings.json")
        return tg
    
    
    def annotate_execution_times_and_results(cls):
        def decorator(func):
            def wrapper(*args, **kwargs):
                start_time = time.time()
                result = func(*args, **kwargs)
                end_time = time.time()
                EX = Namespace("http://example.org/ns#")
                execution_time = end_time - start_time
                execution_node = URIRef(f"http://example.org/execution/{cls.translation.split('#')[-1]}")
    
                cls.ex.add((execution_node, RDF.type, EX.Execution))
                cls.ex.add((execution_node, EX.executionTime, Literal(execution_time, datatype=XSD.decimal)))
                cls.ex.add((execution_node, EX.intermediateResult, Literal(result)))
                print(cls.ex.serialize(format="turtle"))
                return result
            return wrapper
        return decorator
    
    def _translate_and_compile(cls, operation, template_str, context):
        translation = cls.tg.value(subject=operation, predicate=tbox.hasTranslation)   
        cls.translation = translation
        try:
            template = Template(template_str)
            rendered_code = template.render(context)
            fixed_code = autopep8.fix_code(rendered_code) 
        except Exception as e:
            print("Code Generation Error: ", e)
            return None
        
        try:
            compiled_code = compile(fixed_code, '', 'exec')
            new_func = types.FunctionType(compiled_code.co_consts[0], globals(), translation)
            #new_func = cls.annotate_execution_times_and_results()(new_func)  # Apply the decorator
        except Exception as e:
            print("Code Compilation Error: ", e)
            return None
        
        return new_func
    # additional methods here
    

In [6]:
class Operator(metaclass=Implementation):
    """..."""
    
     
    def __init__(self, operation, params):
        
        # Static Metadata
        self.tg = self.__class__.tg
        self.params = params
        # Function Implementaiton
        self.f = self._init_func(operation)
        
        
    def _init_func(self, operation):
        """
        Get the code associated with the operation
        :param operation: operation
        :return: python funcion 
        """
        
        translation = self.tg.value(subject=operation, predicate=tbox.hasTranslation)   

        context = {
            "name": translation.split("#")[1],
            "operator": self.params["operator"],
            "rightOperand": self.params["rightOperand"]
        }
        
        # Implementation Template
        template = """
        def {{ name }}(*args, **kwargs):
            data = kwargs['data'] {{operator}} {{rightOperand}}
            return data
        """
    
        return self.__class__._translate_and_compile(operation, template, context)
    
    def get_func(self):
        return self.f

In [7]:
class Operation(metaclass=Implementation):
    """
    Operation Class
    """
    
    def __init__(self, operation):
        
        # Static Metadata
        self.tg = self.__class__.tg
        # Function Implementaiton
        self.f = self._init_func(operation)
        
        
    def _init_func(self, operation):
        """
        Get the code associated with the operation
        :param operation: operation
        :return: python funcion 
        """
        
        translation = self.tg.value(subject=operation, predicate=tbox.hasTranslation)   
        code = self.tg.value(subject=translation, predicate=tbox.hasCode)
        params = self.tg.objects(subject=translation, predicate=tbox.hasParameters)
        dependencies = self.tg.objects(subject=translation, predicate=tbox.dependsOn)
        params = [self.tg.value(subject=p, predicate=tbox.name) for p in params]
        dependencies = [self.tg.value(subject=d, predicate=tbox.name) for d in dependencies]
                
        c_path = self.tg.value(subject=code, predicate=tbox.code)
        
        codelines = str(c_path).split('\n')
        
        context = {
            "name": translation.split("#")[1],
            "codelines": codelines,
            "params": params,
            "libraries": dependencies
        }
        
        # Implementation Template
        template = """
        def {{ name }}({{",".join(params)}}):
        {% for library in libraries %}
            import {{ library }}
        {% endfor %}
        {% for line in codelines %}
            data = {{  line }}
        {% endfor %}
            return data
        """   
        return self.__class__._translate_and_compile(operation, template, context)
    
    
    def get_func(self):
        return self.f


In [8]:
class PCTranslator:
    """
    Parse the PolicyChecker Operations to an executable Script
    """
    def __init__(self, pc):
        # PolicyChecker
        self.g = pc
        # Execution Graph
        self.eg = Graph()
        
        
    def _validate_graph(self) -> bool:
        """
        Validate the policies grammar is compliant with the grammar defined
        :return: conformance/non-conformance
        """
        from pyshacl import validate
        shapes = Graph().parse("./pc_grammar.ttl", format="turtle")
        conforms, report_graph, report_text = validate(self.g,shacl_graph=shapes)
        #return boolean
        return conforms
         
    def _create_function_as_decorator(self, before_func=None, after_func=None, before_args={}, after_args={}) -> types.FunctionType:
        def decorator(f):
            def wrapper(*args, **kwargs):
                if before_func is not None:
                    before_func(*before_args)
                data = f(*args, **kwargs)
                if after_func is not None:
                    result = after_func(data=data, **after_args)
                    return result
                return data
            return wrapper
        return decorator
    
    
    def _get_func_parameters(self, f):
        import inspect 
        return inspect.signature(f).parameters
    
    def _operation_to_code(self, operation) -> types.FunctionType:
        
        operation_type = str(self.g.value(subject=operation, predicate=RDF.type)).split("#")[-1]
        kwargs = {}
                
        if operation_type == "InitOperation":
            imp = Operation(operation)
            return imp.get_func()
        elif operation_type == "Operation":
            if operation.split("#")[1] == "Constraint":
                operator = self.g.value(subject=operation, predicate=tbox.Operator)
                rightOperand = self.g.value(subject=operation, predicate=tbox.hasRightOperand)
                params = {"operator": operator, "rightOperand": rightOperand} 
                imp = Operator(operation, params)
            else:
                imp = Operation(operation)
                params = self._get_func_parameters(imp.get_func())
                parameters = self.g.objects(subject=operation, predicate=tbox.hasParameter)
                parameters = [str(p).split("#")[1] for p in parameters]

                # TODO: PARAMTERER MATCHING ALGORITHM
                for param in params:
                    if param != "data":
                        if parameters:
                            kwargs[param] = parameters.pop(0)
                
            
            decorated_imp  = self._create_function_as_decorator(after_func=imp.get_func(), after_args=kwargs)
            return decorated_imp
        
   
    def traverse_and_generate(self, language="python") -> types.FunctionType:
        """
        Traverse the policy checker and generate the code
        :param language: 
        :return: 
        """
        # Obtian policy checker
        pc = self.g.value(predicate=RDF.type, object=tbox.PolicyChecker)
        
        # Get first operation
        operation = self.g.value(subject=pc,predicate=tbox.nextStep)
        function = None
        
        # Traverse the policy checker (path)
        while operation:
            operation_type = str(self.g.value(subject=operation, predicate=RDF.type)).split("#")[-1]
            if operation_type == "InitOperation":
                function = self._operation_to_code(operation)
            elif operation_type == "Operation":
                decorator = self._operation_to_code(operation)
                function = decorator(function)            
            operation = self.g.value(subject=operation,predicate=tbox.nextStep)
            
        return function
        
    def translate(self) -> types.FunctionType: 
        """
        Get the policies associated with a data product
        :return: list of policies
        """
        # validate policies
        
        #if self._validate_graph() == True:
            # get policies
        udf = self.traverse_and_generate()
        return udf
        #else:
        #    raise Exception("Policy Checker is not compliant with the grammar")
        

# Translate the Policy Checker

In [9]:
# Translate the Policy Checker 
pc = Graph().parse("/home/acraf/psr/tfm/Fdatavalidation/SideCar/ValidateContract/parser/pc_p2.ttl", format="turtle")
function = PCTranslator(pc).translate()
function

<function __main__.PCTranslator._create_function_as_decorator.<locals>.decorator.<locals>.wrapper(*args, **kwargs)>

In [10]:

function('/home/acraf/psr/tfm/Fdatavalidation/DataProduct/Data/Explotation/UPENN-GBM_clinical_info_v2.1.csv')

True